In [0]:
#Import  packages

import numpy as np
import pandas as pd
import glob
import cv2
import os
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import keras.backend as K 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageOps

# Set random seed for purposes of reproducibility
seed = 21

Using TensorFlow backend.


In [0]:
# Import files from Google drive
from google.colab import drive
drive.mount('/content/drive/')
train_labels_path = '/content/drive/My Drive/train_labels.csv'
test_labels_path = '/content/drive/My Drive/test_labels.csv'
test_images = os.listdir('/content/drive/My Drive/datasets/val_set')
train_images = os.listdir('/content/drive/My Drive/datasets/train_set')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
#Define class labels
classes = ['person','head','hand','foot']

#define lists of columns from train/test csv files
train_labels = pd.read_csv(train_labels_path)
class_ = train_labels.loc[: , "class"]
filenames = train_labels.loc[: , "filename"]
width = train_labels.loc[: , "width"]
height = train_labels.loc[: , "height"]
xmin = train_labels.loc[: , "xmin"]
xmax = train_labels.loc[: , "xmax"]
ymin = train_labels.loc[: , "ymin"]
ymax = train_labels.loc[: , "ymax"]

test_labels = pd.read_csv(test_labels_path)
class_test = test_labels.loc[: , "class"]
filenames_test = test_labels.loc[: , "filename"]
width_test = test_labels.loc[: , "width"]
height_test = test_labels.loc[: , "height"]
xmin_test = test_labels.loc[: , "xmin"]
xmax_test = test_labels.loc[: , "xmax"]
ymin_test = test_labels.loc[: , "ymin"]
ymax_test = test_labels.loc[: , "ymax"]

print(filenames_test[1])

C:\Users\Alexw\OneDrive\Desktop\uni\Applied Machine Learning\Coursework2\datasets\images\test\2008_000223.jpg


In [0]:
#choose dimension for images
input_dim = 228

In [0]:
#Function to create label for a given image.
# Finds largest bounding box in picture and scales the co-ordinates to the desired image size.
# For example, image size (500,500) --> (200,200) , bounding_box (100,200,150,300) --> (40,80,60,120)
# outputs co-ords of bounding box (x_0,y_0,x_1,y_1)

def create_y_label(filenames, xmin, xmax, ymin, ymax, class_, width, height, image_size, file_name,k):
  find_largest_obj = 0
  find_largest_obj_list = [[[]]]
  index = 0
  for i,item in enumerate(filenames):
    if file_name == item:
      x_0 = (xmin[i] / width[i])
      x_1 = (xmax[i] / width[i])
      y_0 = (ymin[i] / height[i])
      y_1 = (ymax[i] / height[i])
      area = (x_1 - x_0)*(y_1 - y_0)
      if area > find_largest_obj:
        find_largest_obj = area
        find_largest_obj_list = [[[x_0,y_0,x_1,y_1]]]
        index = i
  #for j,cls in enumerate(classes):
    #if cls == class_[index]:
      #if j != 0:
        #find_largest_obj_list[5 + j] = 1
  return find_largest_obj_list

In [0]:
# Functions to create Y_train and Y_test by looping create_y_labels for each image in train/test sets

def create_y_train(image_size):
  Y_train = []
  filename_list = []
  new_filename_list = []
  for item in filenames:
    if item not in filename_list:
      filename_list.append(item)
  counter = 0
  for name in filename_list:
    for i,item in enumerate(filenames):
      if name == item:
        if class_[i] == 'person':
          counter += 1
    if counter == 1:
      new_filename_list.append(name)
    counter = 0
  dont_do = [135,137,148,149,152,153,154,161,162,164,165,173,174,176,180,182,185,186,188,190,193,194,200,207,210,211,212,7,69,81,127,5,12,18,24,26,41,47,51,56,68,77,88,95,96,109,115,118,132,141,175,177,183,204,205]
  for k, jpg_image in enumerate(new_filename_list):
    dont_do_switch = False
    for element in dont_do:
      if k == element:
        dont_do_switch = True
    if dont_do_switch == False:
      y_label = create_y_label(filenames, xmin, xmax, ymin, ymax, class_, width, height, image_size, jpg_image,k)
      Y_train.append(y_label)
  Y_train = np.array(Y_train)
  return Y_train

def create_y_test(image_size):
  Y_test = []
  filename_list = []
  new_filename_list = []
  for item in filenames_test:
    if item not in filename_list:
      filename_list.append(item)
  counter = 0
  for name in filename_list:
    for i,item in enumerate(filenames_test):
      if name == item:
        if class_test[i] == 'person':
          counter += 1
    if counter == 1:
      new_filename_list.append(name)
    counter = 0
  dont_do = [80,82,83,86,100,101,102,104,106,111,112,114,116,118,119,120,127,128,132,137,138,139,140,143,145,146,150,152,159,162,164,2,14,21,22,26,30,47,62,64,66,71,75,76,77,109,110,121,133,154,160] # do not include these images.
  for k, jpg_image in enumerate(new_filename_list):
    dont_do_switch = False
    for element in dont_do:
      if k == element:
        dont_do_switch = True
    if dont_do_switch == False:
      y_label = create_y_label(filenames_test, xmin_test, xmax_test, ymin_test, ymax_test, class_test, width_test, height_test, image_size, jpg_image,k)
      Y_test.append(y_label)
  Y_test = np.array(Y_test)
  return Y_test

In [0]:
#Create Y train/test
Y_train = create_y_train(input_dim)
Y_test = create_y_test(input_dim)

#show shape of output labels
print(Y_train.shape)
print(Y_test.shape)



(142, 1, 1, 4)
(114, 1, 1, 4)


In [0]:
#  Functions to create X_train/test.
#   converts images to np array, pads te image to size (500,500) and then scales to desired size
def create_x_train(image_size): 
  X_train = []
  filename_list = []
  new_filename_list = []
  for item in filenames:
    if item not in filename_list:
      filename_list.append(item)
  counter = 0
  for name in filename_list:
    for i,item in enumerate(filenames):
      if name == item:
        if class_[i] == 'person':
          counter += 1
    if counter == 1:
      new_filename_list.append(name)
    counter = 0
  dont_do = [135,137,148,149,152,153,154,161,162,164,165,173,174,176,180,182,185,186,188,190,193,194,200,207,210,211,212,7,69,81,127,5,12,18,24,26,41,47,51,56,68,77,88,95,96,109,115,118,132,141,175,177,183,204,205]
  for k, image_name in enumerate(new_filename_list):
    dont_do_switch = False
    for element in dont_do:
      if k == element:
        dont_do_switch = True
    if dont_do_switch == False:     
      image = Image.open('/content/drive/My Drive/datasets/train_set/' + image_name[-15:])
      sqrWidth = np.ceil(np.sqrt(input_dim*input_dim)).astype(int)
      image = image.resize((sqrWidth, sqrWidth))
      image = np.array(image)
      X_train.append(image)
  X_train = np.array(X_train)
  return X_train


def create_x_test(image_size): 
  X_test = []
  filename_list = []
  new_filename_list = []
  for item in filenames_test:
    if item not in filename_list:
      filename_list.append(item)
  counter = 0
  for name in filename_list:
    for i,item in enumerate(filenames_test):
      if name == item:
        if class_test[i] == 'person':
          counter += 1
    if counter == 1:
      new_filename_list.append(name)
    counter = 0
  dont_do = [80,82,83,86,100,101,102,104,106,111,112,114,116,118,119,120,127,128,132,137,138,139,140,143,145,146,150,152,159,162,164,2,14,21,22,26,30,47,62,64,66,71,75,76,77,109,110,121,133,154,160]
  for k, image_name in enumerate(new_filename_list):
    dont_do_switch = False
    for element in dont_do:
      if k == element:
        dont_do_switch = True
    if dont_do_switch == False:           
      image = Image.open('/content/drive/My Drive/datasets/val_set/' + image_name[-15:])
      sqrWidth = np.ceil(np.sqrt(input_dim*input_dim)).astype(int)
      image = image.resize((sqrWidth, sqrWidth))
      image = np.array(image)
      X_test.append(image)
  X_test = np.array(X_test)
  return X_test

# Create X_train/test
X_train = create_x_train(input_dim)
X_test = create_x_test(input_dim)

#show shape of inputs
print(X_train.shape)
print(X_test.shape)

(142, 228, 228, 3)
(114, 228, 228, 3)


In [0]:
#convert to float and normalize X_train/test

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = np.array(X_train)
X_test = np.array(X_test)
print('done')

done


In [0]:
input_shape = ( input_dim , input_dim , 3 )
dropout_rate = 0.2
alpha = 0.2

def calculate_iou( target_boxes , pred_boxes ):
    xA = K.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = K.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = K.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = K.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = K.maximum( 0.0 , xB - xA ) * K.maximum( 0.0 , yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou


#loss function is a mix of mean squared error and intersection over union.
def custom_loss( y_true , y_pred ):
    mse = tf.losses.mean_squared_error( y_true , y_pred ) 
    iou = calculate_iou( y_true , y_pred ) 
    return mse + ( 1 - iou )

#loss function is a mix of root mean squared error and intersection over union.
def custom_loss_2( y_true , y_pred ):
    rmse = root_mean_squared_error( y_true , y_pred ) 
    iou = calculate_iou( y_true , y_pred ) 
    return rmse + ( 1 - iou )

def iou_metric( y_true , y_pred ):
    return calculate_iou( y_true , y_pred )

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))  

In [0]:
#keras.layers.Dropout(dropout_rate)
model_layers = [       
    keras.layers.Conv2D( 256 , input_shape=( input_dim , input_dim , 3 ) , kernel_size=( 3 , 3 ) , strides=2 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=2 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    #keras.layers.Dropout(dropout_rate),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D( 16 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 16 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),

    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='sigmoid' ),
]

model = keras.Sequential(model_layers)
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss=custom_loss,metrics=[ iou_metric ]) #specify which loss function you want to use and which optimizer.
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_403 (Conv2D)          (None, 113, 113, 256)     7168      
_________________________________________________________________
conv2d_404 (Conv2D)          (None, 56, 56, 256)       590080    
_________________________________________________________________
batch_normalization_169 (Bat (None, 56, 56, 256)       1024      
_________________________________________________________________
conv2d_405 (Conv2D)          (None, 54, 54, 256)       590080    
_________________________________________________________________
conv2d_406 (Conv2D)          (None, 52, 52, 256)       590080    
_________________________________________________________________
batch_normalization_170 (Bat (None, 52, 52, 256)       1024      
_________________________________________________________________
conv2d_407 (Conv2D)          (None, 50, 50, 256)     

In [0]:
model.fit( X_train ,Y_train, epochs=80, validation_data=(X_test , Y_test), batch_size=32)